In [ ]:
import csv
import json
import os
import re
import time

import gym
import roboschool

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from gym import wrappers
from ipywidgets import Video
from multiprocessing import Pool, Process

## Load a Model

Since we use a custom initializer and this gets serialized during the saving process of the model we need to pass it on when we load it again. Unfortunately with the issue of the background TensorFlow session when importing TensorFlow and multiprocessing we cannot define the initializer one time and use it here again. So we define it twice, one inside the create_model() function and here.

In [ ]:
def load_model(model_path):   
    import tensorflow as tf

    class Normc_initializer(tf.keras.initializers.Initializer):
        """
        Create a TensorFlow constant with random numbers normed in the given shape.
        :param std:
        :return:
        """
        def __init__(self, std=1.0):
            self.std = std

        def __call__(self, shape, dtype=None, partition_info=None):
            out = np.random.randn(*shape).astype(np.float32)
            out *= self.std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
            return tf.constant(out)
        
    class ObservationNormalizationLayer(tf.keras.layers.Layer):
        def __init__(self, ob_mean, ob_std, **kwargs):
            self.ob_mean = ob_mean
            self.ob_std = ob_std
            super(ObservationNormalizationLayer, self).__init__(**kwargs)

        def call(self, input):
            return tf.clip_by_value((input - self.ob_mean) / self.ob_std, -5.0, 5.0)
          
        def get_config(self):
            base_config = super(ObservationNormalizationLayer, self).get_config()
            base_config['ob_mean'] = self.ob_mean
            base_config['ob_std'] = self.ob_std
            return base_config
        
        @classmethod
        def from_config(cls, config):
            return cls(**config)
        
    class DiscretizeActionsUniformLayer(tf.keras.layers.Layer):
        def __init__(self, num_ac_bins, adim, ahigh, alow, **kwargs):
            self.num_ac_bins = num_ac_bins
            self.adim = adim
            self.ahigh = ahigh
            self.alow = alow
            super(DiscretizeActionsUniformLayer, self).__init__(**kwargs)

        def call(self, x):            
            # Reshape to [n x i x j] where n is dynamically chosen, i equals action dimension and j equals the number
            # of bins
            scores_nab = tf.reshape(x, [-1, self.adim, self.num_ac_bins])
            # This picks the bin with the greatest value
            a = tf.argmax(scores_nab, 2)
            
            # Then transform the interval from [0, num_ac_bins - 1] to [-1, 1] which equals alow and ahigh
            ac_range_1a = (self.ahigh - self.alow)[None, :]
            return 1. / (self.num_ac_bins - 1.) * tf.keras.backend.cast(a, 'float32') * ac_range_1a + self.alow[None, :]        
        
        # get_config and from_config need to implemented to be able to serialize the model
        def get_config(self):
            base_config = super(DiscretizeActionsUniformLayer, self).get_config()
            base_config['num_ac_bins'] = self.num_ac_bins
            base_config['adim'] = self.adim
            base_config['ahigh'] = self.ahigh
            base_config['alow'] = self.alow
            return base_config
        
        @classmethod
        def from_config(cls, config):
            return cls(**config)
    
    custom_objects = {'Normc_initializer' : Normc_initializer, 
                      'ObservationNormalizationLayer' : ObservationNormalizationLayer,
                      'DiscretizeActionsUniformLayer' : DiscretizeActionsUniformLayer}
    
    return tf.keras.models.load_model(model_path, custom_objects=custom_objects)

def rollout(env, model, render=False, timestep_limit=None, random_stream=None):
    """
    If random_stream is provided, the rollout will take noisy actions with noise drawn from that stream.
    Otherwise, no action noise will be added.
    """

    env_timestep_limit = env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')
    timestep_limit = env_timestep_limit if timestep_limit is None else min(timestep_limit, env_timestep_limit)
    rews = []
    t = 0

    ob = env.reset()
    for _ in range(timestep_limit):
        if render:
            env.render()
        ac = act(ob[None], model, random_stream=random_stream)[0]
        ob, rew, done, _ = env.step(ac)
        rews.append(rew)
        t += 1

        if done:
            break
    return np.array(rews, dtype=np.float32), t

def act(ob, model, random_stream=None):   
    action = model.predict_on_batch(ob)
    
    #if random_stream is not None and model_structure.ac_noise_std != 0:
    #    action += random_stream.randn(*action.shape) * model_structure.ac_noise_std
    
    return action

# Visualize

In [ ]:
def index_main_directory(main_directory):
    index = {}
    for root, dirs, files in os.walk(main_directory):
        if not dirs and 'log.csv' in files and 'config.json' in files:
            index[root] = files
    return index

def load_data(index):
    data = []
    for sub_dir in index:
        log, config, models = None, None, []
        for file in index[sub_dir]:
            if file.endswith('.h5'):
                models.append(file)
            elif file.endswith('log.csv'):
                log = pd.read_csv(os.path.join(sub_dir, file))
            elif file.endswith('config.json'):
                with open(os.path.join(sub_dir, file), encoding='utf-8') as f:
                    config = json.load(f)
        models.sort()
        data.append((log, config, models))
    
    return data

def run_model(env_id, save_directory, model_file_path, record=False):
    env = gym.make(env_id)
    env.reset()
    
    if record:
        video_directory = save_directory + 'videos/' + model_file_path + '/'
        env = wrappers.Monitor(env, video_directory, force=True)
        
    model = load_model(save_directory + model_file_path)
    
    rewards, length = rollout(env, model)
    
    if record:
        return video_directory
    
    return [rewards.sum(), length]

def visualize_model(save_directory, model_file_path, config_file):
                                       
    with Pool(os.cpu_count()) as pool:
        video_directory = pool.apply(func=run_model, args=(config_file['config']['env_id'],
                                                           save_directory,
                                                           model_file_path,
                                                           True))
    
    for file in os.listdir(video_directory):
        if file.endswith('.mp4'):
            return video_directory + file
        
    return None

In [ ]:
def parse_generation_number(model_file_path):
    try: 
        number = int(model_file_path.split('snapshot_')[-1].split('.h5')[0])
        return number
    except ValueError:
        return None       
    
def evaluate_weights(env_id, save_directory, model_file_paths, eval_count=5):
    head_row = ['Generation', 'Eval_per_Gen', 'Eval_Rew_Mean', 'Eval_Rew_Std', 'Eval_Len_Mean']   
        
    for i in range(eval_count):
        head_row.append('Rew_' + str(i))
        head_row.append('Len_' + str(i))
        
    data = []
    
    for model_file_path in model_file_paths:
        results = []
        with Pool(os.cpu_count()) as pool:
            for _ in range(eval_count):
                results.append(pool.apply_async(func=run_model, args=(env_id, 
                                                                      save_directory, 
                                                                      model_file_path)))
                
            for i in range(len(results)):
                results[i] = results[i].get()
        
        rewards = np.array(results)[:, 0]
        lengths = np.array(results)[:, 1]
        
        row = [parse_generation_number(model_file_path),
               eval_count,
               np.mean(rewards),
               np.std(rewards),
               np.mean(lengths)]
        
        assert len(rewards) == len(lengths)
        for i in range(len(rewards)):
            row.append(rewards[i])
            row.append(lengths[i])
        
        data.append(row)
        
    return pd.DataFrame(data, columns = head_row)   

In [ ]:
main_directory = '/home/pdeubel/PycharmProjects/saved-weights/mujoco-halfcheetah-01'

index = index_main_directory(main_directory)

In [ ]:
data = load_data(index)

In [ ]:
#%%javascript
#IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
from IPython.display import display
from IPython.display import JSON
for (log, config, _) in data:
    print(json.dumps(config, indent=4))
    #display(log)
    %matplotlib inline

    plt.plot(log.TimestepsSoFar, log.GenLenMean)
    plt.xlabel('Timesteps')
    plt.ylabel('Cummulative reward')
    plt.show()

In [ ]:
if model_file_paths is not None and config_file is not None:
    evaluation = evaluate_weights(config_file['config']['env_id'], save_directory, model_file_paths, eval_count=5)
else:
    print("Not enough data to evaluate the training. Please provide a directory with enough data.")

In [ ]:
evaluation

In [ ]:
evaluation.to_csv(save_directory + 'evaluation.csv')

In [ ]:
pd.set_option('display.max_columns', None)
data = pd.merge(log, evaluation)
data

In [ ]:
model_file_path = model_file_paths[-1]
video_file = visualize_model(save_directory, model_file_path, config_file)

if video_file is not None:
    video = Video.from_file(video_file)

In [ ]:
%matplotlib inline

plt.plot(data.Generation, data.Eval_Rew_Mean)
plt.xlabel('Generation')
plt.ylabel('Cummulative reward')
plt.show()

plt.plot(data.TimestepsSoFar, data.Eval_Rew_Mean)
plt.xlabel('Timesteps')
plt.ylabel('Cummulative reward')
plt.show()